In [1]:
%load_ext autoreload
%autoreload 2

In [115]:
import torch
import torch.nn as nn
import torch.nn.functional as F

########################
### Hyper-paramaters ###
########################
M = 2
batch_size = 6
dim = 3

#######################
### Generated Input ###
#######################
labels = torch.tensor([0, 1, 1, 5, 5, 5])
# label_list = [labels for _ in range(3)]
# labels = torch.cat(label_list, dim=-1)

pred_list, prob_list = [], []
for i in range(0, M, 1):
    torch.manual_seed(i)
    pred = torch.randn((batch_size, dim))
    pred_list.append(pred)
    prob_list.append(F.softmax(pred, dim=1))
print(f"prob_list (len={len(prob_list)}, shape={prob_list[0].shape})")
for prob in prob_list:
    print(prob)
print('')
print(f"labels (shape={labels.shape})")
print(labels, end='\n\n')

probs = torch.stack(prob_list, dim=0)
print(f"probs ({probs.shape})", end='\n\n')

prob_list (len=2, shape=torch.Size([6, 3]))
tensor([[0.1350, 0.1315, 0.7335],
        [0.5212, 0.4290, 0.0498],
        [0.0818, 0.7343, 0.1838],
        [0.2949, 0.2903, 0.4148],
        [0.3497, 0.4258, 0.2245],
        [0.0663, 0.3912, 0.5426]])
tensor([[0.0805, 0.1747, 0.7448],
        [0.2504, 0.4876, 0.2620],
        [0.1196, 0.5203, 0.3601],
        [0.9043, 0.0297, 0.0660],
        [0.5180, 0.2627, 0.2193],
        [0.3216, 0.1651, 0.5133]])

labels (shape=torch.Size([6]))
tensor([0, 1, 1, 5, 5, 5])

probs (torch.Size([2, 6, 3]))



[autoreload of mmdet.models.losses.ai28.contrastive_loss_plus failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/ws/external/mmdet/models/losses/ai28/contrastive_loss_plus.py", line 69, in <module>
    class ContrastiveLossPlus(nn.Module):
  File "/opt/conda/lib/python3.7/site-packages/mmcv/util

In [142]:
import torch
import torch.nn as nn
import torch.nn.functional as F

########################
### Hyper-paramaters ###
########################
M = 3
batch_size = 6
dim = 10
eps = 1e-7

#######################
### Generated Input ###
#######################
labels = torch.tensor([0, 1, 1, 5, 5, 5])

pred_list, prob_list = [], []
for i in range(0, M, 1):
    torch.manual_seed(i)
    pred = torch.randn((batch_size, dim))
    pred_list.append(pred)
print(f"pred_list (len={len(pred_list)}, shape={pred_list[0].shape})")
print(f"labels (shape={labels.shape})", end='\n\n')


logits_clean, logits_aug1, logits_aug2 = F.normalize(pred_list[0], dim=1), \
                                         F.normalize(pred_list[1], dim=1), \
                                         F.normalize(pred_list[2], dim=1),

probs = torch.stack([logits_clean, logits_aug1, logits_aug2], dim=0) # (M, B, D)
probs = probs.reshape(-1, probs.shape[-1]) # (M*B, D)
labels = labels.repeat(3) # (M,)
print(f"probs ({probs.shape})")
print(f"labels ({labels.shape})", end='\n\n')
print('----------------')

weighted_kld_list = []
for i in range(len(probs)):
    # Anchor
    anchor = probs[i]
    # print(f"1. anchor ({anchor.shape})", end='\n\n')

    # Mixture of positives (with same label)
    y_mask = torch.eq(labels, labels[i])
    # print(f"2.1. y_mask ({y_mask.shape})")
    num_pos = torch.sum(y_mask)
    # print(f"2.2. num_pos={num_pos}")
    y_mask = y_mask.unsqueeze(-1).expand(probs.size())
    # print(f"2.3. y_mask ({y_mask.shape})", end='\n\n')

    mixture = torch.sum(probs * y_mask, dim=0) / num_pos
    # print(f"2.4. mixture ({mixture.shape}) = {torch.sum(mixture)}", end='\n\n')

    # Calculate JSD
    weighted_kld = kl_div(mixture.clamp(min=eps).log().unsqueeze(0),
                          anchor.unsqueeze(0),
                          reduction = 'sum') / num_pos
    weighted_kld_list.append(weighted_kld)

weighted_kld = torch.stack(weighted_kld_list, dim=0)
jsd = torch.sum(weighted_kld)
print(jsd)

pred_list (len=3, shape=torch.Size([6, 10]))
labels (shape=torch.Size([6]))

probs (torch.Size([18, 10]))
labels (torch.Size([18]))

----------------
tensor(16.6264)
